# Pyrsistent

> Persistent/Immutable/Functional data structures for Python

[pyrsistent](https://github.com/tobgu/pyrsistent)

## PVector (similiar to a list)

In [1]:
from pyrsistent import v, pvector

In [2]:
# No mutation of vectors once created, instead they
# are "evolved" leaving the original untouched

v1 = v(1, 2, 3)
v1

pvector([1, 2, 3])

In [3]:
v2 = v1.append(4)
v2

pvector([1, 2, 3, 4])

In [4]:
v3 = v2.set(1, 5)
v3

pvector([1, 5, 3, 4])

In [5]:
# Random access and slicing

v3[1]

5

In [6]:
v3[1:3]

pvector([5, 3])

In [7]:
# Iteration

list(x + 1 for x in v3)

[2, 6, 4, 5]

In [8]:
pvector(2 * x for x in range(3))

pvector([0, 2, 4])

## PMap (similiar to dict)

In [9]:
from pyrsistent import m, pmap, v

In [10]:
# No mutation of maps once created, instead they are
# "evolved" leaving the original untouched

m1 = m(a=1, b=2)

In [11]:
m1

pmap({'a': 1, 'b': 2})

In [12]:
m2 = m1.set('c', 3)
m2

pmap({'c': 3, 'a': 1, 'b': 2})

In [13]:
m3 = m2.set('a', 5)
m3

pmap({'c': 3, 'a': 5, 'b': 2})

In [14]:
m3['a']

5

In [15]:
# Evolution of nested persistent structures

m4 = m(a=5, b=6, c=v(1, 2))
m4.transform(('c', 1), 17)

pmap({'a': 5, 'b': 6, 'c': pvector([1, 17])})

In [16]:
m5 = m(a=1, b=2)
m5

pmap({'a': 1, 'b': 2})

In [17]:
# Evolve by merging with other mappings

m5.update(m(a=2, c=3), {'a': 17, 'd': 35})

pmap({'c': 3, 'a': 17, 'b': 2, 'd': 35})

In [18]:
pmap({'x': 1, 'y': 2}) + pmap({'y': 3, 'z': 4})

pmap({'y': 3, 'x': 1, 'z': 4})

In [19]:
# Dict-like methods to convert to list and iterate

m3.items()

pvector([('c', 3), ('a', 5), ('b', 2)])

In [20]:
list(m3)

['c', 'a', 'b']

## PSet (similiar to set)

In [21]:
from pyrsistent import s

In [22]:
# No mutation of sets once created, you know the story...

s1 = s(1, 2, 3, 2)
s1

pset([1, 2, 3])

In [23]:
s2 = s1.add(4)
s2

pset([1, 2, 3, 4])

In [24]:
s3 = s1.remove(1)
s3

pset([2, 3])

In [25]:
# Full support for set operations

s1 | s(3, 4, 5)

pset([1, 2, 3, 4, 5])

In [26]:
s1 & s(3, 4, 5)

pset([3])

In [27]:
s1 < s2

True

In [28]:
s1 < s(3, 4, 5)

False

## PRecord (PMap on steroids with fixed fields, optional type and invariant checking)

In [29]:
from pyrsistent import PRecord, field

In [30]:
class ARecord(PRecord):
    x = field()
r = ARecord(x=3)
r

ARecord(x=3)

In [31]:
r.x

3

In [32]:
r.set(x=2)

ARecord(x=2)

In [33]:
r.set(y=2)

AttributeError: 'y' is not among the specified fields for ARecord

In [34]:
# type information

class BRecord(PRecord):
    x = field(type=int)
    y = field(type=(int, type(None)))

In [35]:
BRecord(x=3, y=None)

BRecord(y=None, x=3)

In [36]:
BRecord(x=3.0)

PTypeError: Invalid type for field BRecord.x, was float

In [37]:
# mandatory fields

from pyrsistent import InvariantException
class CRecord(PRecord):
    x = field(mandatory=True)
r = CRecord(x=3)
try:
   r.discard('x')
except InvariantException as e:
   print(e.missing_fields)

('CRecord.x',)


In [38]:
# invariants

class RestrictedVector(PRecord):
    __invariant__ = lambda r: (r.y >= r.x, 'x larger than y')
    x = field(invariant=lambda x: (x > 0, 'x negative'))
    y = field(invariant=lambda y: (y > 0, 'y negative'))
r = RestrictedVector(y=3, x=2)
try:
   r.set(x=-1, y=-2)
except InvariantException as e:
   print(e.invariant_errors)

('x negative', 'y negative')


In [39]:
try:
   r.set(x=2, y=1)
except InvariantException as e:
   print(e.invariant_errors)

('x larger than y',)


In [40]:
# invariants with multiple assertions

class EvenX(PRecord):
    x = field(invariant=lambda x: ((x > 0, 'x negative'), (x % 2 == 0, 'x odd')))
try:
   EvenX(x=-1)
except InvariantException as e:
   print(e.invariant_errors)

(('x negative', 'x odd'),)


In [41]:
# factory functions for fields

class DRecord(PRecord):
    x = field(factory=int)

class ERecord(PRecord):
    d = field(type=DRecord)

ERecord.create({'d': {'x': '1'}})

ERecord(d=DRecord(x=1))

In [42]:
# collection fields

from pyrsistent import pset_field, pmap_field, pvector_field

class MultiRecord(PRecord):
    set_of_ints = pset_field(int)
    map_int_to_str = pmap_field(int, str)
    vector_of_strs = pvector_field(str)

In [43]:
# serialization back to dicts

from datetime import date

class Person(PRecord):
    name = field(type=str)
    birth_date = field(type=date,
                       serializer=lambda format, d: d.strftime(format['date']))

john = Person(name=u'John', birth_date=date(1985, 10, 21))
john.serialize({'date': '%Y-%m-%d'})

{'name': 'John', 'birth_date': '1985-10-21'}

## PClass (class, fixed fields, optional type and invariant checking)

In [44]:
from pyrsistent import PClass, field
class AClass(PClass):
    x = field()
a = AClass(x=3)
a

AClass(x=3)

In [45]:
a.x

3

In [46]:
# checked collections

from pyrsistent import CheckedPVector, CheckedPMap, CheckedPSet, thaw

class Positives(CheckedPSet):
    __type__ = (int, int)
    __invariant__ = lambda n: (n >= 0, 'Negative')

class Lottery(PRecord):
    name = field(type=str)
    numbers = field(type=Positives, invariant=lambda p: (len(p) > 0, 'No numbers'))

class Lotteries(CheckedPVector):
    __type__ = Lottery

class LotteriesByDate(CheckedPMap):
    __key_type__ = date
    __value_type__ = Lotteries

lotteries = LotteriesByDate.create({date(2015, 2, 15): [{'name': 'SuperLotto', 'numbers': {1, 2, 3}},
                                                        {'name': 'MegaLotto',  'numbers': {4, 5, 6}}],
                                    date(2015, 2, 16): [{'name': 'SuperLotto', 'numbers': {3, 2, 1}},
                                                        {'name': 'MegaLotto',  'numbers': {6, 5, 4}}]})
lotteries

LotteriesByDate({datetime.date(2015, 2, 16): Lotteries([Lottery(name='SuperLotto', numbers=Positives([1, 2, 3])), Lottery(name='MegaLotto', numbers=Positives([4, 5, 6]))]), datetime.date(2015, 2, 15): Lotteries([Lottery(name='SuperLotto', numbers=Positives([1, 2, 3])), Lottery(name='MegaLotto', numbers=Positives([4, 5, 6]))])})

In [47]:
# The checked versions support all operations that the corresponding
# unchecked types do

lottery_0215 = lotteries[date(2015, 2, 15)]
lottery_0215.transform([0, 'name'], 'SuperDuperLotto')

Lotteries([Lottery(name='SuperDuperLotto', numbers=Positives([1, 2, 3])), Lottery(name='MegaLotto', numbers=Positives([4, 5, 6]))])

In [48]:
# But also makes asserts that types and invariants hold

lottery_0215.transform([0, 'name'], 999)

PTypeError: Invalid type for field Lottery.name, was int

In [49]:
lottery_0215.transform([0, 'numbers'], set())

InvariantException: Field invariant failed, invariant_errors=[No numbers], missing_fields=[]

In [50]:
# They can be converted back to python built ins with either thaw()
# or serialize() (which provides possibilities to customize serialization)

thaw(lottery_0215)

[{'name': 'SuperLotto', 'numbers': {1, 2, 3}},
 {'name': 'MegaLotto', 'numbers': {4, 5, 6}}]

In [51]:
lottery_0215.serialize()

[{'name': 'SuperLotto', 'numbers': {1, 2, 3}},
 {'name': 'MegaLotto', 'numbers': {4, 5, 6}}]

## Transformations

In [52]:
# Basic examples

from pyrsistent import inc, freeze, thaw, rex, ny, discard

v1 = freeze([1, 2, 3, 4, 5])
v1.transform([2], inc)

pvector([1, 2, 4, 4, 5])

In [53]:
v1.transform([lambda ix: 0 < ix < 4], 8)

pvector([1, 8, 8, 8, 5])

In [54]:
v1.transform([lambda ix, v: ix == 0 or v == 5], 0)

pvector([0, 2, 3, 4, 0])

In [55]:
# The (a)ny matcher can be used to match anything

v1.transform([ny], 8)

pvector([8, 8, 8, 8, 8])

In [56]:
# Regular expressions can be used for matching

scores = freeze({'John': 12, 'Joseph': 34, 'Sara': 23})
scores.transform([rex('^Jo')], 0)

pmap({'Joseph': 0, 'John': 0, 'Sara': 23})

In [57]:
# Transformations can be done on arbitrarily deep structures

news_paper = freeze({'articles': [{'author': 'Sara', 'content': 'A short article'},
                                  {'author': 'Steve', 'content': 'A slightly longer article'}],
                     'weather': {'temperature': '11C', 'wind': '5m/s'}})
short_news = news_paper.transform(['articles', ny, 'content'], lambda c: c[:25] + '...' if len(c) > 25 else c)
very_short_news = news_paper.transform(['articles', ny, 'content'], lambda c: c[:15] + '...' if len(c) > 15 else c)
very_short_news.articles[0].content

'A short article'

In [58]:
very_short_news.articles[1].content

'A slightly long...'

In [59]:
# When nothing has been transformed the original data structure is kept

short_news is news_paper

True

In [60]:
very_short_news is news_paper

False

In [61]:
very_short_news.articles[0] is news_paper.articles[0]

True

In [62]:
# There is a special transformation that can be used to discard elements. Also
# multiple transformations can be applied in one call

thaw(news_paper.transform(['weather'], discard, ['articles', ny, 'content'], discard))

{'articles': [{'author': 'Sara'}, {'author': 'Steve'}]}

## Evolvers (Mutable view of the underlying persistent data structure with “transaction like” semantics for PVector, PMap and PSet.)

In [63]:
from pyrsistent import v

# In place mutation as when working with the built in counterpart

v1 = v(1, 2, 3)
e = v1.evolver()
e[1] = 22
e = e.append(4)
e = e.extend([5, 6])
e[5] += 1
len(e)

6

In [64]:
# The evolver is considered *dirty* when it contains changes compared to the underlying vector

e.is_dirty()

True

In [65]:
# But the underlying pvector still remains untouched

v1

pvector([1, 2, 3])

In [66]:
# Once satisfied with the updates you can produce a new pvector containing the updates.
# The new pvector will share data with the original pvector in the same way that would have
# been done if only using operations on the pvector.
v2 = e.persistent()
v2

pvector([1, 22, 3, 4, 5, 7])

In [67]:
# The evolver is now no longer considered *dirty* as it contains no differences compared to the
# pvector just produced.
e.is_dirty()

False

In [68]:
# You may continue to work with the same evolver without affecting the content of v2

e[0] = 11

In [69]:
# Or create a new evolver from v2. The two evolvers can be updated independently but will both
# share data with v2 where possible.

e2 = v2.evolver()
e2[0] = 1111
e.persistent()

pvector([11, 22, 3, 4, 5, 7])

In [70]:
e2.persistent()

pvector([1111, 22, 3, 4, 5, 7])

## Thraw and freeze (Conversion between immutable and mutable world.)

In [71]:
from pyrsistent import freeze, thaw, v, m

freeze([1, {'a': 3}])

pvector([1, pmap({'a': 3})])

In [72]:
thaw(v(1, m(a=3)))

[1, {'a': 3}]